<img src="https://raw.githubusercontent.com/Ironhack-Data-Madrid-Marzo-2021/W7-Kaggle_competition/master/images/PORTADA.jpg">

# Import libraries

In [1]:
import numpy as np
import pandas as pd
import src.limpieza as lm
import pickle
from sklearn import metrics

# Import Data

In [2]:
test = pd.read_csv("data/test_limpio_s.csv")
train = pd.read_csv("data/train_limpio_s.csv")

In [3]:
test = pd.read_csv("data/test_limpio.csv")
train = pd.read_csv("data/train_limpio.csv")

# Create target and feature columns

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = train.drop("price", axis=1)
y = train["price"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
X_train.dtypes

Unnamed: 0      int64
id              int64
carat         float64
cut             int64
color           int64
clarity       float64
depth         float64
table         float64
dtype: object

# Random Forest 

In [8]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
rfr = RandomForestRegressor()

# Parameter Tuning
Now that we have found the most efficient model, let's try the different `hyperparameters` to improve it.

In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
best

NameError: name 'best' is not defined

In [ ]:
parameters = {
 'max_depth': [30,50,70],
 'min_samples_leaf': [ 2, 5,7],
 'min_samples_split': [80, 100, 150],
 'n_estimators': [400,500,600]}

In [25]:
parameters = {'bootstrap': [True],
 'max_depth': [65,75],
 'max_features': ['auto'],
 'min_samples_leaf': [4,6],
 'min_samples_split': [20,50],
 'n_estimators': [500,600]}

In [33]:
parameters = {'bootstrap': [True],
 'max_depth': [80,90],
 'max_features': ['auto'],
 'min_samples_leaf': [2,3],
 'min_samples_split': [15,20],
 'n_estimators': [450,550]}

In [34]:
gs = GridSearchCV(estimator = rfr,
                  param_grid = parameters,
                 cv = 3,
                 verbose = 3,
                 scoring = "neg_mean_squared_error",
                 return_train_score=True)

In [35]:
gs.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=450; total time=  25.0s
[CV 2/3] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=450; total time=  24.3s
[CV 3/3] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=450; total time=  23.6s
[CV 1/3] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=550; total time=  28.8s
[CV 2/3] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=550; total time=  29.3s
[CV 3/3] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=15, n_estimators=550; total time=  29.0s
[CV 1/3] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True], 'max_depth': [80, 90],
                         'max_features': ['auto'], 'min_samples_leaf': [2, 3],
                         'min_samples_split': [15, 20],
                         'n_estimators': [450, 550]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

# Hora ==> 16:40

In [41]:
gs_results = pd.DataFrame(gs.cv_results_)
gs_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bootstrap,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,...,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,23.715171,0.509603,0.602577,0.037966,True,80,auto,2,15,450,...,-0.011280,-0.011099,-0.011146,0.000096,4,-0.006115,-0.006118,-0.006149,-0.006127,0.000015
1,28.335183,0.191667,0.713911,0.022506,True,80,auto,2,15,550,...,-0.011257,-0.011105,-0.011142,0.000083,3,-0.006123,-0.006113,-0.006158,-0.006131,0.000019
2,22.355505,0.450224,0.565370,0.030257,True,80,auto,2,20,450,...,-0.011350,-0.011188,-0.011230,0.000086,10,-0.007030,-0.006998,-0.007072,-0.007033,0.000030
3,28.987895,0.746515,0.730784,0.042524,True,80,auto,2,20,550,...,-0.011346,-0.011188,-0.011228,0.000085,9,-0.007020,-0.007001,-0.007067,-0.007029,0.000028
4,23.169062,0.198014,0.580861,0.002408,True,80,auto,3,15,450,...,-0.011297,-0.011119,-0.011172,0.000089,6,-0.006475,-0.006468,-0.006503,-0.006482,0.000015


In [42]:
gs_results = gs_results[["param_bootstrap","param_max_depth","param_max_features","param_min_samples_leaf","param_min_samples_split","param_n_estimators","mean_test_score","mean_train_score"]]

In [43]:
gs_results.sort_values("mean_test_score", ascending = False).head(10)

,param_bootstrap,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,mean_test_score,mean_train_score
8,True,90,auto,2,15,450,-0.011133,-0.006128
9,True,90,auto,2,15,550,-0.011134,-0.006131
1,True,80,auto,2,15,550,-0.011142,-0.006131
0,True,80,auto,2,15,450,-0.011146,-0.006127
12,True,90,auto,3,15,450,-0.011166,-0.006478
4,True,80,auto,3,15,450,-0.011172,-0.006482
5,True,80,auto,3,15,550,-0.011175,-0.006478
13,True,90,auto,3,15,550,-0.011178,-0.006481
3,True,80,auto,2,20,550,-0.011228,-0.007029
2,True,80,auto,2,20,450,-0.011230,-0.007033


In [44]:
best = gs.best_estimator_
best

RandomForestRegressor(max_depth=90, min_samples_leaf=2, min_samples_split=15,
                      n_estimators=450)

In [45]:
metrics.mean_squared_error(best.predict(X_test),y_test)

0.010564121464047409

In [46]:
pickle.dump(best,open("param/rf_b3_00923", "wb"))